In [ ]:
!nvidia-smi

Mon Jul 15 12:22:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -q transformers flash_attn timm einops supervision==0.22.0rc1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.9/126.9 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 60.7 MB/s eta 0:00:00


# Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Extracting Zip File

In [ ]:
import shutil
import os
def extract_zip(zip_file, extract_to):
    try:
        shutil.unpack_archive(zip_file, extract_to, 'zip')
        print("Extraction complete.")
    except Exception as e:
        print(f"Error: {e}")

zip_file = "/content/drive/MyDrive/data_test.zip"
extract_to = "/content/"

# Check if the zip file exists
if not os.path.exists(zip_file):
    print("Error: The specified ZIP file does not exist.")
    exit()

    # Check if the extract directory exists, if not, create it
if not os.path.exists(extract_to):
    os.makedirs(extract_to)

extract_zip(zip_file, extract_to)

Extraction complete.


## Importing Florence 2 Model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor

# CHECKPOINT = "microsoft/Florence-2-base-ft"
CHECKPOINT = "microsoft/Florence-2-large-ft"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(CHECKPOINT, trust_remote_code=True).to(DEVICE)
processor = AutoProcessor.from_pretrained(CHECKPOINT, trust_remote_code=True)

# To generate Labels

In [ ]:
import os
import numpy as np
from PIL import Image
import supervision as sv

# Assuming `processor` and `model` are already defined and initialized, and `DEVICE` is set.

def generate_detections(image):
    text = "<bee>"
    task = "<OD>"

    # Process the image with the model
    inputs = processor(text=text, images=image, return_tensors="pt").to(DEVICE)
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        num_beams=3
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    response = processor.post_process_generation(generated_text, task=task, image_size=image.size)

    # Get detections from the response
    detections = sv.Detections.from_lmm(sv.LMM.FLORENCE_2, response, resolution_wh=image.size)

    return detections

def detections_to_yolov5txt(detections, image_width, image_height):
    class_labels = ['bee']
    if detections is None:
        raise ValueError("No detections found.")

    yolov5txt_lines = []

    for i in range(len(detections.xyxy)):
        x1, y1, x2, y2 = detections.xyxy[i]
        class_name = detections.data['class_name'][i] if isinstance(detections.data['class_name'][i], str) else 'Unknown'

        class_index = class_labels.index(class_name)
        bbox_width = x2 - x1
        bbox_height = y2 - y1
        x_center = x1 + bbox_width / 2
        y_center = y1 + bbox_height / 2

        normalized_x_center = x_center / image_width
        normalized_y_center = y_center / image_height
        normalized_bbox_width = bbox_width / image_width
        normalized_bbox_height = bbox_height / image_height

        line = f"{class_index} {normalized_x_center:.6f} {normalized_y_center:.6f} {normalized_bbox_width:.6f} {normalized_bbox_height:.6f}"
        yolov5txt_lines.append(line)

    return yolov5txt_lines

def process_detections(detections, image_size, output_file):
    if detections is None:
        print("No detections found.")
        return
    image_width, image_height = image_size[0], image_size[1]

    yolov5txt_lines = detections_to_yolov5txt(detections, image_width, image_height)

    with open(output_file, 'w') as f:
        for line in yolov5txt_lines:
            f.write(line + "\n")

def main(dataset_path):
    for filename in os.listdir(dataset_path):
        file_path = os.path.join(dataset_path, filename)
        if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            with Image.open(file_path) as img:
                image_size = img.size  # (width, height)

                # Generate detections for the current image
                detections = generate_detections(img)

                output_file = os.path.join(dataset_path, f"{os.path.splitext(filename)[0]}_labels.txt")
                process_detections(detections, image_size, output_file)

if __name__ == "__main__":
    dataset_path = "/content/data_test"
    main(dataset_path)


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

def read_labels(label_path):
    with open(label_path, 'r') as file:
        labels = file.readlines()
    return [[float(x) if i else x for i, x in enumerate(label.strip().split())] for label in labels] # Keep the first element (class label) as a string, convert the rest to floats

def draw_bounding_boxes(image_path, label_path, output_path=None):
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to read image from {image_path}")
        return

    image_height, image_width, _ = image.shape

    # Read labels
    labels = read_labels(label_path)

    for label in labels:
        class_id, x_center, y_center, width, height = label # Now class_id is a string and the rest are floats

        # Convert normalized coordinates to pixel coordinates
        x_center *= image_width
        y_center *= image_height
        width *= image_width
        height *= image_height

        x_min = int(x_center - width / 2)
        y_min = int(y_center - height / 2)
        x_max = int(x_center + width / 2)
        y_max = int(y_center + height / 2)

        # Draw bounding box
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

    # Save or display the image with bounding boxes
    if output_path:
        # Ensure the directory exists
        output_dir = os.path.dirname(output_path)
        if not os.path.exists(output_dir):
            print(f"Directory does not exist, creating: {output_dir}")
            os.makedirs(output_dir)

        success = cv2.imwrite(output_path, image)
        if success:
            print(f"Output image saved to {output_path}")
        else:
            print(f"Error: Unable to save image to {output_path}")
    else:
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()

# Example usage
image_path = '/content/data_test/inaturalist-17855827-12390505-ind9.jpeg'
label_path = '/content/data_test/inaturalist-17855827-12390505-ind9_labels.txt'
output_path = '/content/bbox/output_image_with_boxes.jpg'  # Use absolute path here
draw_bounding_boxes(image_path, label_path, output_path)

Directory does not exist, creating: /content/bbox
Output image saved to /content/bbox/output_image_with_boxes.jpg


# To download only labels folder

In [ ]:
import shutil

dataset_path = "/content/data_test"
labels_path = os.path.join(dataset_path, "labels")

# Zip the directory
shutil.make_archive(labels_path, 'zip', labels_path)


'/content/data_test/labels.zip'

In [ ]:
from google.colab import files

files.download(labels_path + '.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# To download Entire Dataset

In [ ]:
    dataset_zip_path = shutil.make_archive(dataset_path, 'zip', dataset_path)

    # Download the zip file
    files.download(dataset_zip_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Only GUI

In [ ]:
!pip install gradio
import gradio as gr
from google.colab import files

def auto_annotate(input_folder, output_folder):
    if not input_folder or not output_folder:
        return "Please select both input and output folders."
    # Add backend code for auto annotation here
    return f"Auto annotating images from {input_folder} to {output_folder}"

def manual_annotate(input_folder, output_folder):
    if not input_folder or not output_folder:
        return "Please select both input and output folders."
    # Add backend code for manual annotation here
    return f"Manual annotating images from {input_folder} to {output_folder}"

def browse_folder():
    uploaded = files.upload()
    for fn in uploaded.keys():
        return fn

with gr.Blocks() as demo:
    gr.Markdown("## Image Annotation Tool")

    with gr.Tab("Auto Annotate"):
        input_folder_auto = gr.Textbox(label="Input Folder")
        output_folder_auto = gr.Textbox(label="Output Folder")
        auto_button = gr.Button("Auto Annotate")
        auto_output = gr.Textbox(label="Output")
        auto_button.click(fn=auto_annotate, inputs=[input_folder_auto, output_folder_auto], outputs=auto_output)

    with gr.Tab("Manual Annotate"):
        input_folder_manual = gr.Textbox(label="Input Folder")
        output_folder_manual = gr.Textbox(label="Output Folder")
        manual_button = gr.Button("Manual Annotate")
        manual_output = gr.Textbox(label="Output")
        manual_button.click(fn=manual_annotate, inputs=[input_folder_manual, output_folder_manual], outputs=manual_output)

    with gr.Tab("Browse Folders"):
        browse_button = gr.Button("Browse Folder")
        browse_output = gr.Textbox(label="Selected Folder")
        browse_button.click(fn=browse_folder, inputs=[], outputs=browse_output)

demo.launch()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.4 M

# GUI with backend implemented

In [ ]:
!pip install einops timm flash_attn
!pip install gradio torch transformers pillow supervision
import torch
from transformers import AutoModelForCausalLM, AutoProcessor
import os
import numpy as np
from PIL import Image
import supervision as sv
import gradio as gr
from google.colab import files

# Load Florence 2 model and processor
CHECKPOINT = "microsoft/Florence-2-large-ft"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(CHECKPOINT, trust_remote_code=True).to(DEVICE)
processor = AutoProcessor.from_pretrained(CHECKPOINT, trust_remote_code=True)

def generate_detections(image):
    text = "<bee>"
    task = "<OD>"

    inputs = processor(text=text, images=image, return_tensors="pt").to(DEVICE)
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        num_beams=3
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    response = processor.post_process_generation(generated_text, task=task, image_size=image.size)

    detections = sv.Detections.from_lmm(sv.LMM.FLORENCE_2, response, resolution_wh=image.size)
    return detections

def detections_to_yolov5txt(detections, image_width, image_height):
    class_labels = ['bee']
    yolov5txt_lines = []

    for i in range(len(detections.xyxy)):
        x1, y1, x2, y2 = detections.xyxy[i]
        class_name = detections.data['class_name'][i] if isinstance(detections.data['class_name'][i], str) else 'Unknown'

        class_index = class_labels.index(class_name)
        bbox_width = x2 - x1
        bbox_height = y2 - y1
        x_center = x1 + bbox_width / 2
        y_center = y1 + bbox_height / 2

        normalized_x_center = x_center / image_width
        normalized_y_center = y_center / image_height
        normalized_bbox_width = bbox_width / image_width
        normalized_bbox_height = bbox_height / image_height

        line = f"{class_index} {normalized_x_center:.6f} {normalized_y_center:.6f} {normalized_bbox_width:.6f} {normalized_bbox_height:.6f}"
        yolov5txt_lines.append(line)

    return yolov5txt_lines

def process_detections(detections, image_size, output_file):
    if detections is None:
        print("No detections found.")
        return
    image_width, image_height = image_size

    yolov5txt_lines = detections_to_yolov5txt(detections, image_width, image_height)

    with open(output_file, 'w') as f:
        for line in yolov5txt_lines:
            f.write(line + "\n")

def auto_annotate(input_folder, output_folder):
    if not input_folder or not output_folder:
        return "Please select both input and output folders."

    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)
        if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            with Image.open(file_path) as img:
                image_size = img.size  # (width, height)

                # Generate detections for the current image
                detections = generate_detections(img)

                output_file = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_labels.txt")
                process_detections(detections, image_size, output_file)

    return f"Auto annotation completed. Files saved to {output_folder}."

def manual_annotate(input_folder, output_folder):
    # Add manual annotation logic here, similar to auto_annotate
    return "Manual annotation is currently not implemented."

def browse_folder():
    uploaded = files.upload()
    for fn in uploaded.keys():
        return fn

with gr.Blocks() as demo:
    gr.Markdown("## Image Annotation Tool")

    with gr.Tab("Auto Annotate"):
        input_folder_auto = gr.Textbox(label="Input Folder")
        output_folder_auto = gr.Textbox(label="Output Folder")
        auto_button = gr.Button("Auto Annotate")
        auto_output = gr.Textbox(label="Output")
        auto_button.click(fn=auto_annotate, inputs=[input_folder_auto, output_folder_auto], outputs=auto_output)

    with gr.Tab("Manual Annotate"):
        input_folder_manual = gr.Textbox(label="Input Folder")
        output_folder_manual = gr.Textbox(label="Output Folder")
        manual_button = gr.Button("Manual Annotate")
        manual_output = gr.Textbox(label="Output")
        manual_button.click(fn=manual_annotate, inputs=[input_folder_manual, output_folder_manual], outputs=manual_output)

    with gr.Tab("Browse Folders"):
        browse_button = gr.Button("Browse Folder")
        browse_output = gr.Textbox(label="Selected Folder")
        browse_button.click(fn=browse_folder, inputs=[], outputs=browse_output)

demo.launch()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 73.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cuspa

config.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large-ft:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large-ft:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7c5c666f99859fc7ad.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import cv2
import os

# Constants
WINDOW_NAME = "Image Annotation Tool"
TRACKBAR_IMG = "Image"
TRACKBAR_CLASS = "Class"
LINE_THICKNESS = 2

# List of class names for annotation
CLASS_LIST = ["bee", "apis", "bombus"]  # Modify as needed

# Initialize global variables
img_index = 0
class_index = 0
mouse_x = 0
mouse_y = 0
drawing = False
is_bbox_selected = False
img_objects = []
annotations = []

def decrease_index(current_index, last_index):
    current_index -= 1
    if current_index < 0:
        current_index = last_index
    return current_index

def increase_index(current_index, last_index):
    current_index += 1
    if current_index > last_index:
        current_index = 0
    return current_index

def draw_line(img, x, y, height, width, color):
    cv2.line(img, (x, 0), (x, height), color, LINE_THICKNESS)
    cv2.line(img, (0, y), (width, y), color, LINE_THICKNESS)

def yolo_format(class_index, point_1, point_2, width, height):
    x_center = float((point_1[0] + point_2[0]) / (2.0 * width))
    y_center = float((point_1[1] + point_2[1]) / (2.0 * height))
    x_width = float(abs(point_2[0] - point_1[0])) / width
    y_height = float(abs(point_2[1] - point_1[1])) / height
    items = map(str, [class_index, x_center, y_center, x_width, y_height])
    return ' '.join(items)

def append_bb(ann_path, line):
    with open(ann_path, 'a') as myfile:
        myfile.write(line + '\n')

def mouse_callback(event, x, y, flags, param):
    global mouse_x, mouse_y, drawing, img, img_copy, point_1, point_2

    mouse_x, mouse_y = x, y

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        point_1 = (x, y)
        point_2 = (x, y)

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img.copy()
            cv2.rectangle(img_copy, point_1, (x, y), (0, 255, 0), LINE_THICKNESS)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        point_2 = (x, y)
        annotations.append((class_index, point_1, point_2))
        cv2.rectangle(img, point_1, point_2, (0, 255, 0), LINE_THICKNESS)

def set_img_index(val):
    global img_index
    img_index = val

def set_class_index(val):
    global class_index
    class_index = val

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append((filename, img))
    return images

def main():
    global img, img_copy, img_index, class_index

    # Load images from the 'images' folder
    images = load_images_from_folder('images')  # Change to your image directory
    if not images:
        print("No images found in the specified folder.")
        return

    num_images = len(images)
    last_img_index = num_images - 1
    last_class_index = len(CLASS_LIST) - 1

    # Create a window
    cv2.namedWindow(WINDOW_NAME)
    cv2.setMouseCallback(WINDOW_NAME, mouse_callback)

    # Create trackbars
    cv2.createTrackbar(TRACKBAR_IMG, WINDOW_NAME, 0, last_img_index, set_img_index)
    cv2.createTrackbar(TRACKBAR_CLASS, WINDOW_NAME, 0, last_class_index, set_class_index)

    while True:
        filename, img = images[img_index]
        img_copy = img.copy()
        height, width = img.shape[:2]

        # Show image
        cv2.imshow(WINDOW_NAME, img_copy)
        pressed_key = cv2.waitKey(1)

        if pressed_key == ord('d'):  # Next image
            img_index = increase_index(img_index, last_img_index)
            cv2.setTrackbarPos(TRACKBAR_IMG, WINDOW_NAME, img_index)
        elif pressed_key == ord('a'):  # Previous image
            img_index = decrease_index(img_index, last_img_index)
            cv2.setTrackbarPos(TRACKBAR_IMG, WINDOW_NAME, img_index)
        elif pressed_key == ord('w'):  # Next class
            class_index = increase_index(class_index, last_class_index)
            cv2.setTrackbarPos(TRACKBAR_CLASS, WINDOW_NAME, class_index)
        elif pressed_key == ord('s'):  # Previous class
            class_index = decrease_index(class_index, last_class_index)
            cv2.setTrackbarPos(TRACKBAR_CLASS, WINDOW_NAME, class_index)
        elif pressed_key == ord('q'):  # Quit
            break

    cv2.destroyAllWindows()

    # Save annotations
    for filename, img in images:
        ann_path = os.path.splitext(filename)[0] + '.txt'
        for class_idx, pt1, pt2 in annotations:
            line = yolo_format(class_idx, pt1, pt2, width, height)
            append_bb(ann_path, line)

if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'images'

In [ ]:
!pip install einops timm flash_attn
!pip install gradio torch transformers pillow supervision
import torch
from transformers import AutoModelForCausalLM, AutoProcessor
import os
import numpy as np
from PIL import Image
import supervision as sv
import gradio as gr

# Checkpoint for the model
CHECKPOINT = "microsoft/Florence-2-large-ft"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model and processor
model = AutoModelForCausalLM.from_pretrained(CHECKPOINT, trust_remote_code=True).to(DEVICE)
processor = AutoProcessor.from_pretrained(CHECKPOINT, trust_remote_code=True)

def generate_detections(image):
    text = "<bee>"
    task = "<OD>"

    inputs = processor(text=text, images=image, return_tensors="pt").to(DEVICE)
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        num_beams=3
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    response = processor.post_process_generation(generated_text, task=task, image_size=image.size)

    detections = sv.Detections.from_lmm(sv.LMM.FLORENCE_2, response, resolution_wh=image.size)
    return detections

def detections_to_yolov5txt(detections, image_width, image_height):
    class_labels = ['bee']
    yolov5txt_lines = []

    for i in range(len(detections.xyxy)):
        x1, y1, x2, y2 = detections.xyxy[i]
        class_name = detections.data['class_name'][i] if isinstance(detections.data['class_name'][i], str) else 'Unknown'

        class_index = class_labels.index(class_name)
        bbox_width = x2 - x1
        bbox_height = y2 - y1
        x_center = x1 + bbox_width / 2
        y_center = y1 + bbox_height / 2

        normalized_x_center = x_center / image_width
        normalized_y_center = y_center / image_height
        normalized_bbox_width = bbox_width / image_width
        normalized_bbox_height = bbox_height / image_height

        line = f"{class_index} {normalized_x_center:.6f} {normalized_y_center:.6f} {normalized_bbox_width:.6f} {normalized_bbox_height:.6f}"
        yolov5txt_lines.append(line)

    return yolov5txt_lines

def process_detections(detections, image_size, output_file):
    if detections is None:
        print("No detections found.")
        return
    image_width, image_height = image_size

    yolov5txt_lines = detections_to_yolov5txt(detections, image_width, image_height)

    with open(output_file, 'w') as f:
        for line in yolov5txt_lines:
            f.write(line + "\n")

def auto_annotate(input_folder, output_folder):
    if not input_folder or not output_folder:
        return "Please select both input and output folders."

    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)
        if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            with Image.open(file_path) as img:
                image_size = img.size  # (width, height)

                # Generate detections for the current image
                detections = generate_detections(img)

                output_file = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_labels.txt")
                process_detections(detections, image_size, output_file)

    return f"Auto annotation completed. Files saved to {output_folder}."

def manual_annotate(input_folder, output_folder):
    # Add manual annotation logic here, similar to auto_annotate
    return "Manual annotation is currently not implemented."

with gr.Blocks() as demo:
    gr.Markdown("## Image Annotation Tool")

    with gr.Tab("Auto Annotate"):
        input_folder_auto = gr.Textbox(label="Input Folder")
        output_folder_auto = gr.Textbox(label="Output Folder")
        auto_button = gr.Button("Auto Annotate")
        auto_output = gr.Textbox(label="Output")
        auto_button.click(fn=auto_annotate, inputs=[input_folder_auto, output_folder_auto], outputs=auto_output)

    with gr.Tab("Manual Annotate"):
        input_folder_manual = gr.Textbox(label="Input Folder")
        output_folder_manual = gr.Textbox(label="Output Folder")
        manual_button = gr.Button("Manual Annotate")
        manual_output = gr.Textbox(label="Output")
        manual_button.click(fn=manual_annotate, inputs=[input_folder_manual, output_folder_manual], outputs=manual_output)

demo.launch()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cuspa

config.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large-ft:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large-ft:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5ba31026aa92a5910f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
